<a href="https://colab.research.google.com/github/codeantik/Auto-Encoders/blob/main/AutoEncoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AutoEncoders

##Downloading the dataset

###ML-100K

In [22]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2023-06-04 11:55:09--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.2’

ml-100k.zip.2       100%[===================>]   4.70M  2.84MB/s    in 1.7s    

2023-06-04 11:55:11 (2.84 MB/s) - ‘ml-100k.zip.2’ saved [4924029/4924029]

Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: ml-100k      ml-100k.zip.1  ml-1m      ml-1m.zip.1
ml-100k.zip  ml-100k.zip.2  ml-1m.zip  sample_data


###ML-1M

In [23]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2023-06-04 11:56:28--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.2’

ml-1m.zip.2         100%[===================>]   5.64M  3.39MB/s    in 1.7s    

2023-06-04 11:56:30 (3.39 MB/s) - ‘ml-1m.zip.2’ saved [5917549/5917549]

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: ml-100k      ml-100k.zip.1  ml-1m      ml-1m.zip.1  sample_data
ml-100k.zip  ml-100k.zip.2  ml-1m.zip  ml-1m.zip.2


##Importing the libraries

In [25]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [26]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [27]:
print(movies)

         0                                   1                             2
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
...    ...                                 ...                           ...
3878  3948             Meet the Parents (2000)                        Comedy
3879  3949          Requiem for a Dream (2000)                         Drama
3880  3950                    Tigerland (2000)                         Drama
3881  3951             Two Family House (2000)                         Drama
3882  3952               Contender, The (2000)                Drama|Thriller

[3883 rows x 3 columns]


In [28]:
print(users)

         0  1   2   3      4
0        1  F   1  10  48067
1        2  M  56  16  70072
2        3  M  25  15  55117
3        4  M  45   7  02460
4        5  M  25  20  55455
...    ... ..  ..  ..    ...
6035  6036  F  25  15  32603
6036  6037  F  45   1  76006
6037  6038  F  56   1  14706
6038  6039  F  45   0  01060
6039  6040  M  25   6  11106

[6040 rows x 5 columns]


In [29]:
print(ratings)

            0     1  2          3
0           1  1193  5  978300760
1           1   661  3  978302109
2           1   914  3  978301968
3           1  3408  4  978300275
4           1  2355  5  978824291
...       ...   ... ..        ...
1000204  6040  1091  1  956716541
1000205  6040  1094  5  956704887
1000206  6040   562  5  956704746
1000207  6040  1096  4  956715648
1000208  6040  1097  4  956715569

[1000209 rows x 4 columns]


## Preparing the training set and the test set


In [30]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')

test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [31]:
print(training_set)

[[        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


In [32]:
print(test_set)

[[        1        10         3 875693118]
 [        1        12         5 878542960]
 [        1        14         5 874965706]
 ...
 [      459       934         3 879563639]
 [      460        10         3 882912371]
 [      462       682         5 886365231]]


## Getting the number of users and movies


In [33]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [34]:
print(nb_users, nb_movies)

943 1682


## Converting the data into an array with users in lines and movies in columns


In [35]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [37]:
print(training_set[0])

[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0, 2.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 2.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 4.0, 5.0, 0.0, 2.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 4.0, 3.0, 5.0, 1.0, 3.0, 0.0, 3.0, 2.0, 0.0, 4.0, 0.0, 4.0, 3.0, 0.0, 2.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 5.0, 2.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 5.0, 3.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 5.0, 0.0, 5.0, 5.0, 4.0, 0.0,

In [38]:
print(test_set[0])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 4.0, 0.0, 3.0, 3.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 3.0, 0.0, 5.0, 4.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 4.0, 3.0, 1.0, 0.0, 4.0, 0.0, 1.0, 0.0, 4.0, 5.0, 5.0, 0.0, 4.0, 3.0, 5.0, 0.0, 0.0, 0.0, 4.0, 5.0, 3.0, 0.0, 0.0, 0.0, 5.0, 3.0, 4.0, 0.0, 5.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 1.0, 5.0, 5.0, 0.0, 0.0, 3.0, 3.0, 0.0, 1.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 5.0, 3.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 5.0, 4.0, 0.0, 0.0, 5.0, 2.0, 0.0, 4.0, 0.0, 3.0, 4.0, 4.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 4.0, 4.0, 4.0, 0.0, 3.0, 3.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 3.0,

## Converting the data into Torch tensors

In [39]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [40]:
print(training_set)

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])


In [41]:
print(test_set)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


## Creating the architecture of the Neural Network


In [42]:
class SAE(nn.Module):

  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()
  
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x
  
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [43]:
print(training_set.shape)

torch.Size([943, 1682])


## Training the SAE


In [44]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data * mean_corrector)
      s += 1.
      optimizer.step()
  print(f'epoch: {epoch} loss: {train_loss / s}')

epoch: 1 loss: 1.7713518142700195
epoch: 2 loss: 1.0968583822250366
epoch: 3 loss: 1.0533199310302734
epoch: 4 loss: 1.038332462310791
epoch: 5 loss: 1.030870795249939
epoch: 6 loss: 1.0266056060791016
epoch: 7 loss: 1.0237318277359009
epoch: 8 loss: 1.0218098163604736
epoch: 9 loss: 1.0207346677780151
epoch: 10 loss: 1.0195987224578857
epoch: 11 loss: 1.0186314582824707
epoch: 12 loss: 1.018431305885315
epoch: 13 loss: 1.0180083513259888
epoch: 14 loss: 1.0174428224563599
epoch: 15 loss: 1.0168768167495728
epoch: 16 loss: 1.016945481300354
epoch: 17 loss: 1.016672968864441
epoch: 18 loss: 1.0164836645126343
epoch: 19 loss: 1.0163607597351074
epoch: 20 loss: 1.0161420106887817
epoch: 21 loss: 1.0159794092178345
epoch: 22 loss: 1.0160737037658691
epoch: 23 loss: 1.0157029628753662
epoch: 24 loss: 1.0158637762069702
epoch: 25 loss: 1.0157467126846313
epoch: 26 loss: 1.0155608654022217
epoch: 27 loss: 1.0152360200881958
epoch: 28 loss: 1.0150002241134644
epoch: 29 loss: 1.0126124620437622

## Testing the SAE


In [45]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data * mean_corrector)
    s += 1.
print(f'test_loss: {test_loss / s}')

test_loss: 0.9491337537765503
